In [1]:
from docx import *
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
# document = Document("Converted/pdf2go.docx")
document = Document("Converted/pdf2go-wps.docx")

In [3]:
for paragraph in document.paragraphs:
    
    # first line indent (segitiga atas di ruler)
    first = paragraph.paragraph_format.first_line_indent
    
    # left indent (segitiga bawah di ruler)
    left = paragraph.paragraph_format.left_indent
    
    print(first,left,paragraph.text+"\n")

None None A

None 0 

None 0 

None 0 

None 0 

None None 

-152400 227965 a and A I /a/ the ¤rst letter of the Latin alphabet used for writing Indonesian.

-152400 227965 a II (abbr) are (a unit of square measure in the metric system, equal to 100 square meters or about 119.6 square yards).

0 75565 a III (abbr) ampere.

0 75565 a IV ahh (hesitation).

-152400 227965 A V (in music) the sixth tone/note in the ascending scale of C major.

None 75565 A VI car license plate for Banten.

0 75565 à and @ at.

0 75565 – Rp 1.000,- at Rp 1,000.00.

None 75565 A ’45 [Angkatan ’45] the Generation of 1945.

None 75565 AA car license plate for Kedu (Magelang).

0 75565 aah ah.

0 75565 –, itu kan proyék swasta.

0 75565 Ah, that’s a private project.

0 75565 aala and a’ala dynasty; > RAJAKULA.

0 75565 – Tang Tang dynasty.

0 75565 aam (A) general.

0 75565 Rois – General Chairman (of the NU); > AM.

0 75565 aau ow! ouch!

0 75565 ab I (A ob) father.

0 75565 ab II (ob) a tin opium box/cylinder.

-76835 229235 penuh ~ dedicated.

-76835 229235 ~ masyarakat public service.

-76835 229235 2 servitude, submission, submissiveness.

-76835 229235 ~ pekara- ngan/tanah encumbrance with servitude.

-76835 229235 ~ pekarangan jalan right of way.

-76835 229235 3 devotion, dedication.

0 76835 abdidalemisme serfdom, servitude.

0 76835 abdikasi (D) abdication.

0 76835 abdis (D) abbess.

0 76835 abdomén (D/E) abdomen.

0 76835 abdominal (D/E) abdominal.

-152400 228600 abdu (A) (from abdi, element used in proper names connected with one of the 99 names given to Allah) 1 slave.

-152400 228600 2 servant.

-152400 228600 Abdullah (the Servant of God); Abdullah bin Abdulkadir Munsyi the name of the pioneer of modern Malay literature in the early 19th century; born in Malacca of mixed Malay­Indian parentage; Malay language teacher (= munsyi) and clerk of Sir Thomas Stamford Raf¶es (1811–1816), the ¤rst and only British Governor General of the Netherlands Indies.

-152400 228600 Abdul Ghafur/

None 76200 abu III defeated, to lose (in certain games).

-19050 76200 abu-abu northern blue¤n tuna, long­tailed tuna, Neothunnus rarus.

-19050 76200 abuan 1 part of a rice paddy yield made available to sawah workers; ration, share.

-19050 76200 2 a nest egg (money saved for an emergency).

-152400 None Abubakar I (A) one of the ¤rst companions of the Prophet Muhammad.

0 76200 abubakar II (joc acr) [atas budi baik Golkar] by Golkar’s goodwill.

0 76200 abudemén (J) > ABONEMÉN.

0 76200 abuh I (Jv) 1 swelling, in¶ammation.

0 76200 2 swollen.

0 76200 sakit – a) dropsy.

-109855 337820 dropsical.

-109855 337820 c) edema.

0 152400 abuhan to suffer from dropsy.

-76200 228600 abuh-abuhan 1 swelling, in¶ammation.

-76200 228600 2 swollen.

-76200 228600 sakit ~ a) dropsy.

-76200 228600 b) dropsical.

0 76200 abuh II noisy; busy.

0 76200 abui > ABOI.

0 76200 abuk I dust.

0 76200 – bunga pollen.

0 76200 – gergaji sawdust.

0 76200 abuk II (M) hair.

-153035 228600 abuk III (Jv) men

0 0 mengacung to point/move upward.

-76200 229235 mengacungi to point out, draw attention to.

-76200 229235 ~ jempol to applaud, acclaim, praise.

-76200 229235 Meréka patut diacungi jempol.

-76200 229235 They should be praised.

-76200 228600 
mengacungkan [and ngacungin (J coq)] 1 to put up (one’s hand to greet or pray).

-76200 228600 2 to raise, lift, hold up (esp one’s hand), point

0 228600 s.t.

0 228600 ~ ibu jarinya to give the thumbs­up sign, indicate that s.t.

0 228600 is good/well done.

0 228600 ~ jari berbentuk huruf V and ~ dua jari mem- bentuk huruf V to give a V for victory sign.

0 228600 ~ jari satu to give the thumbs­up sign.

0 228600 ~ jari dua to give the V for victory sign.

0 228600 ~ jari tiga to make a fuck­you sign (by making a ¤st and placing the thumb between the index and middle ¤ngers).

0 228600 ~ jarinya to raise one’s hand (in classroom).

0 228600 ~ jempol to give the thumbs­up sign.

0 228600 Itu patut diacungkan jempol.

0 228600 We should take

0 152400 ketika dia ~ di Jakarta

0 228600 …, when he was/lived in Jakarta ...

0 228600 ~ pada jalan yang tepat to be on the right track.

0 228600   ~ di hadapan saya, notaris (in notarial

None None language)  in my presence, notary public.

None None 2 to be well off,

0 228600 well to do, rich.

0 228600 kurang ~ lacking, de¤cient, scant.

0 228600 kekurang- beradaan lack, de¤ciency, scantiness, shortage.

0 228600 keberadaan 1 presence, where s.o.

0 228600 is, whereabouts, existence; creation (of s.t.).

0 228600 2 wealth.

None 152400 ada-berada there is s.t.

None 152400 hidden (such as a reason/intention, etc.).

0 228600 tentu ada-beradanya there must be some reason behind it.

-76200 227965 mengadakan [and ngadain (J coq)] 1 to organize, arrange, give, have, hold, make, deliver, issue.

-76200 227965 ~ améndemén to make/introduce an amendment.

-76200 227965 ~ dengar-pendapat to hold a hearing.

-76200 227965 ~ diskusi/pembahasan to hold discussions.

-76200 227965 ~ drop( 

...

-152400 228600 – ganti kerugian property and casualty insurance.

-152400 228600 – jiwa(sraya) life insurance.

-152400 228600 – jiwa kumpulan group life insurance.

-152400 228600 – kebakaran ¤re insurance.

-152400 228600 – kecelakaan (diri) ( personal) accident insurance.

-152400 228600 – kelompok group insurance.

-152400 228600 – kematian life insurance.

0 228600 
– kendaraan bermotor motor vehicle/ automobile insurance.

0 228600 – kerugian indemnity insurance.

0 228600 – keséhatan health insurance.

0 228600 – krédit jiwa credit life insurance.

0 228600 – kurang underinsurance.

0 228600 – laut marine insurance.

0 228600 – mobil car insurance.

0 228600 – pemasangan instalasi bangunan construction all­risks insurance.

0 228600 – pembangunan ge- dung contractor’s all­risks insurance.

0 228600 – pencurian theft insurance.

0 228600 – pengangkutan cargo insurance.

0 228600 – pengangkutan uang cash­in­transit insurance.

0 228600 – perjalanan travel insurance.

0 228600 – pi

0 75565 atlas III (D/E) atlas, i.e., the upper vertebra of the neck.

0 75565 atlét (D) athlete.

0 75565 atlétik (E) and atlétis (D) 1 athletic.

0 75565 tubuh – an athlete’s body.

0 75565 2

None None track and ¤eld (sports).

0 76200 atlétis (D) athletic.

-152400 228600 atlit > ATLÉT.

-152400 228600 – angkat besi weight lifter.

-152400 228600 – binaraga > BINARAGAWAN.

0 76200 ATM (init) ATM.

0 76200 atma(n) (Skr) 1 soul.

0 76200 2 breath.

-152400 None Atmani Wedhana freshmen of the Akadémi Kepolisian [Akpol]; meaning: “with the attitude of a ¤ghting patriot, ready to sacri¤ce himself in serving unsel¤shly.”

None 76200 Atmawan a male student of Atma Jaya University.

None 76200 Atmawati a female student of Atma Jaya University.

None 76200 Atmil [Atasé Militér] Military Attaché.

0 76200 atmométer (D/E) atmometer.

0 76200 atmos¤r (D/E) atmosphere.

-152400 None ATMR [Aktiva Tertambang Menurut Risiko] Risk Weighted Assets.

0 76200 atob > ATOP.

0 76200 atok – moyang ancesto

0 76200 autokrat (D/E) autocrat.

0 76200 autologi (D/E) autology.

0 76200 automobil (D/E) automobile, (motor)car.

0 76200 automotif (D/E) automotive.

0 76200 autonomi (D/E) autonomy; > OTONOMI.

0 76835 autopsi (D) autopsy; > OTOPSI.

-76200 229235 mengautopsi and melakukan – to conduct/carry out an autopsy/ post­mortem.

0 76835 autoskop (D/E) autoscope.

0 76835 autropik (D/E) autropic.

0 76835 auzubillah > AUDZUBILLAH.

-152400 229235 avaang a tree producing oil/minyak tengkawang, butter tree, Sho- rea spp., mainly found in Kalimantan; > TENGKAWANG.

None 76835 aval (D) guarantee (of a bill).

0 153035 mengaval(kan) to guarantee (a bill).

0 76835 avalis (D) guarantor, backer, accommodation party.

0 76835 avarai, avarij and avary (D leg) marine damage.

0 76835 avérsi (D) aversion.

0 76835 avés (ob naut) to heave up.

None 76835 avgas [aviation gasoline] (D) gasoline for aircraft.

None 76835 AVI [Aangifte van Inlading] Shipment Declaration (in customs).

0 76835 aviasi (D) a

0 75565 awasendi (Skr neo) mengawasendikan to dislocate.

0 151765 pengawasendian dislocation.

0 75565 awasenjata mengawasenjatakan to disarm.

0 75565 awaserap pengawaserapan desorption.

0 75565 awasuara (Skr neo) devoiced.

0 75565 awasumbat unclogged.

76200 75565 pengawasumbatan unclogging.

76200 75565 awat I (ob) (= apa buat?) why? awat II mengawat to hold, grasp.

76200 75565 awat III unplowed ground.

76200 75565 awatara (Skr) incarnation, avatar.

76200 75565 awatata (Skr neo) disorder.

-152400 227965 awat-awat (M) almost not.

-152400 227965 – sampai almost not reaching its destination (as of a voice).

-76200 151765 awatular decontaminated.

-76200 151765 mengawatularkan to decontaminate.

-76200 151765 pengawatularan decontamination.

0 75565 awau gibbon; > UNGKA.

-76200 151765 awawarna (Skr neo) decolorized.

-76200 151765 mengawawarnakan to decolorize.

-76200 151765 pengawawarnaan decolorization.

0 75565 awda (Br) 1 you.

0 75565 2 your; > ANDA I.

0 76200 awé (Jv) 

-152400 228600 2 (Jv) untroubled, comfortable.

-152400 228600 ayeng-ayengan (J/Jv) 1 to go back and forth.

-152400 228600 2 to roam around, go round and round in circles.

0 76200 ayér (coq) > AIR.

None 76200 ayid (Jv) 1 sticky, mucous, slimy.

None 76200 2 slippery (of an eel).

-152400 229235 ayim legal term for a woman with no husband (unmarried or widow).

0 76835 ayir (ob) tuna ¤sh; > TONGKOL i.

None 76835 ayn (A) name of the 18th letter of the Arabic alphabet.

-153035 229235 ayo(h) and ayok come on! let’s ...! Sudah –! OK, let’s go! –lah du- luan go ahead ¤rst.

0 153035 mengayo-ayo to keep on saying “Come on!” to s.o.

None 76835 ayom (Jv) shaded, protected, guarded, sheltered.

0 153035 mengayomi to protect, shelter, shade, take care of.

0 153035 diayomi oléh

None 229235 under the aegis of.

0 153035 terayomi protected, sheltered.

0 153035 ayoman shelter.

-76200 229235 pengayom 1 protector.

-76200 229235 2 patron.

-76200 229235 berpengayom under the auspices/aegis of

In [4]:
# Target yang di crawling
# [lema, sublema, gabungan(-,~), similar(panah,"and"), keterangan, akronim([]), terjemahan]

# CARA NGEDAPETIN (bantu isi cara ngedapetin atau ciri khas atau langkah2 buat ngedapetin datanya)
# [v] == sudah dibuat codingannya

# [v] LEMA : indent pertama,bold (simpan di tempLema)
# [v] LEMA (jika sebaris dg lema lainnya) : indent pertama, jika kondisi tempLema isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempLema
# [v] LEMA (jika sebaris dg sublema) : indent kedua, jika kondisi tempSublema harus isi, jika tidak bisa distemming dan huruf awal kata sama seperti awal kata tempSublema

# [v] SUBLEMA : indent kedua,deteksi bold (simpan di tempSublema)
# [v] SUBLEMA (jika sebaris dg lema): indent pertama, jika kondisi tempLema isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempLema
# [v] SUBLEMA (jika sebaris dg sublema lainnya) : indent kedua, jika kondisi tempSublema harus isi, jika bisa distemming dan huruf awal kata tidak sama seperti awal kata tempSublema

# [v] GABUNGAN(-,~) : deteksi kata italic, ambil kata-katanya simpan ke list tempGabungan, cek di list ada - atau ~ tidak, kalau ada berarti itu kata gabungan dan replace - dan ~ dengan lema/sublema 

# [v] SIMILAR ("and") : indent pertama/kedua, deteksi bold, jika kata sebelumnya adalah and maka similar
# SIMILAR (panah) :

# [v] KETERANGAN :

# [v] AKRONIM :

# TERJEMAHAN :

In [5]:
# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan]
    
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*|^\)|^\)\s*|^\w\)\s*|^\s*|^\s|^\s\w|;|;\s|-\s","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("s.t.","something",x[idx])
        x[idx] = re.sub("s.o.'s","someone's",x[idx])
        x[idx] = re.sub("s.o","someone",x[idx])
        x[idx] = re.sub("q.v.","which see (refer to that word)",x[idx])
        x[idx] = re.sub("k.o.","kind(s) of",x[idx])
        x[idx] = re.sub("o.s.","oneself",x[idx])
        x[idx] = re.sub("e.o.","eachother",x[idx])
        x[idx] = re.sub("\.$|\s\.$|\.\s$|\.\s","",x[idx])
  
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","keterangan","terjemahan"])
    
    return x,y

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan) :     
    if word == "opp" :
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if paragraph.runs[index-1].text == "(":

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

        if re.search("\)", paragraph.runs[index+1].text) :
            keterangan = " ".join(tempKeterangan)
            tempKeterangan = []

    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if re.search("\)", paragraph.runs[index+1].text) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
    else :
        tempGabungan.append(word)
    
    return keterangan,tempKeterangan,tempGabungan

In [6]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","keterangan","terjemahan"])
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
list_and = ["and", " and", "and ", " and ", "[and", "[and "]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod"]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
keterangan = ""
terjemahan = ""
tempLemaSublema = ""
tempTerjemahan = ""
    
boolLemaSublema = False
boolPanah = False
tempGabungan = []
tempKeterangan = []
tempAkronim = []
    
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600 
        
        #baca indent lema
        if (((first == None or first == 0) and left <= 76835) or ((first == -152400 or first == -153035) and left >= 227965)) :      
            
            tempLeft = left 
            
            #deteksi kata bold
            if(run.bold) and run.text.isspace() == False :
                
                split_text = run.text.split()
                word = split_text[0]

                if word.isdigit() == False  and word not in roman_numerals and word != "/":
                    
                    #LEMA
                    #jika lema kosong atau belum terdapat lema, lema = word
                    if boolLemaSublema == False :
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        keterangan = ""
                        terjemahan = ""

#                         boolLemaSublema = False
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        lema = word
                        tempLemaSublema = word
                        boolLemaSublema = True
                        
                    #jika lema tidak kosong atau variabel lema sudah terisi
                    else :
                        
                        #SIMILAR (AND)
                        #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                        if paragraph.runs[index-1].text in list_and or paragraph.runs[index-2].text in list_and :
                            similar = word
                            
                        #LEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA YANG LAIN)    
                        # mengatasi jika dalam satu paragraf ada > 1 bold
                        elif stemmer.stem(word.lower()) == word.lower() and word[0].lower() == tempLemaSublema[0].lower() :
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":        
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            lema = word
                            tempLemaSublema = word
                            boolLemaSublema = True
                            
                        #jika bukan similar
                        else :
                            
                            #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                            #jika sublema kosong
                            if sublema == "" :
                                if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                    x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                    print(x[0])
                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                                sublema = word
                                
                                lema = ""
                                similar = ""
                                gabungan = ""
                                similar = ""
                                panah = ""
                                akronim = ""
                                antonim = ""
                                keterangan = ""
                                terjemahan = ""
                                
                            #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                            else :
                                if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                    x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                    print(x[0])
                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                                sublema = word
                                
                                lema = ""
                                similar = ""
                                gabungan = ""
                                similar = ""
                                panah = ""
                                akronim = ""
                                antonim = ""
                                keterangan = ""
                                terjemahan = ""
            
            #deteksi kata italic (kata gabungan, keterangan)
            elif(run.italic) and run.text.isspace() == False :

                results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan)
            
                keterangan = results[0]
                tempKeterangan = results[1]
                tempGabungan = results[2]  
                
#                 print(tempGabungan)

            # deteksi kata reguler (akronim, terjemahan)
            elif run.text.isspace() == False :
                
                word = run.text
                
                #GABUNGAN
                if "–" in tempGabungan or "~" in tempGabungan:
                    gabungan = " ".join(tempGabungan)
                    gabungan = re.sub("–|~",tempLemaSublema,gabungan)
                    gabungan = re.sub("  "," ", gabungan)

                    tempGabungan = []
                
                else :
                    if len(tempGabungan) != 0:

                        if boolPanah == True and panah != word and word not in roman_numerals :
                            gabungan = " ".join(tempGabungan)
                            panah = panah+" "+gabungan
                            
                            gabungan = ""
                            tempGabungan = []
                            
                        elif index > 1 and len(tempGabungan) <= 3:
                            gabungan = " ".join(tempGabungan)

                            terjemahan = terjemahan+" "+gabungan

                            gabungan = ""
                            tempGabungan = []

                        else :
                            gabungan = " ".join(tempGabungan)
                            gabungan = re.sub(r'  ',' ', gabungan)

                            tempGabungan = []
                
                #AKRONIM
                # mengambil akronim
                if paragraph.runs[index-1].text == "[" and word not in list_and:
                    tempAkronim.append(word)

                    if re.search("\]", paragraph.runs[index+1].text) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

                    if re.search("\]", paragraph.runs[index+1].text) :
                        akronim = " ".join(tempAkronim)
                        akronim = re.sub(r'  ',' ', akronim)
                        tempAkronim = []
                else :
                    #PANAH
                    if paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">") :
                                
                        re.sub("\s","",word)
                        panah = word
                        boolPanah = True
                        
                    elif paragraph.runs[index-1].italic == True and (paragraph.runs[index-1].text == "opp" or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)
                        
                        antonim = word
                        
                    else :

                        #TERJEMAHAN
                        if word == "(" and paragraph.runs[index+1].italic == True or word == ")" and paragraph.runs[index-1].italic == True :
                            word = ""

#                         if re.search("\)\s", word) or re.search("\w\)\s", word)  and paragraph.runs[index-1].italic == True :
#                             word = re.sub("\)\s|\w\)\s","",word)
                        
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if word in list_and and (paragraph.runs[index-1].bold == True and (paragraph.runs[index+1].bold == True or paragraph.runs[index+2].bold == True)):
                            word = ""

                        if word == "[" or word == "]" or word == ">":
                            word = ""

                        if terjemahan == "" and word not in roman_numerals :
                            terjemahan = word
                        elif terjemahan != "" and word not in roman_numerals :
                            terjemahan = terjemahan+" "+word
                        
                        if re.search("\.$|\s\.$|\.\s$", terjemahan) and terjemahan != ".":
                            
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)

                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            keterangan = ""
                            terjemahan = ""
                    
                    if boolPanah == True and panah != word and word not in roman_numerals :
                        panah = panah+" "+word
                        
                    if boolPanah == True and re.search("\.", word) :
                        panah = re.sub("\.|\s.","",panah)
                        boolPanah = False
                        
        #baca indent sublema
        else :
            tempLeft = left 
            
            #deteksi kata bold
            if(run.bold) and run.text.isspace() == False :
                
                split_text = run.text.split()
                word = split_text[0]

                if word.isdigit() == False  and word not in roman_numerals and word != "/":

                    #SUBLEMA
                    #jika sublema kosong
                    if boolLemaSublema == False :
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        keterangan = ""
                        terjemahan = ""

#                         boolLemaSublema = False
                        boolPanah = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []
                        
                        sublema = word
                        tempLemaSublema = word
                        boolLemaSublema = True
                        
                        keterangan = ""
                        akronim = ""
                                
                    #jika sublema tidak kosong
                    else :
                        
                        #SIMILAR (AND)
                        #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                        if paragraph.runs[index-1].text in list_and or paragraph.runs[index-2].text in list_and :
                            similar = word
                           
                        #LEMA (JIKA SEBARIS/SEPARAGRAF DENGAN SUBLEMA)
                        # mengatasi jika dalam satu paragraf ada > 1 bold
                        elif stemmer.stem(word.lower()) == word.lower() and word[0].lower() != tempLemaSublema[0].lower() :
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            lema = word
                            tempLemaSublema = word
                            boolLemaSublema = True
                        
                        #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                        #jika bukan, maka simpan sublema ke list terlebih dahulu
                        else :
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            sublema = word
                            tempLemaSublema = word
                            boolLemaSublema = True

                            lema = ""
                            similar = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            keterangan = ""
                            terjemahan = ""
    
            #deteksi kata italic (kata gabungan, keterangan)
            elif(run.italic) and run.text.isspace() == False :
                
                results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan)

                keterangan = results[0]
                tempKeterangan = results[1]
                tempGabungan = results[2]
                
#                 print(tempGabungan)

            # deteksi kata reguler (akronim, terjemahan)
            elif run.text.isspace() == False :   
                word = run.text
                
                #GABUNGAN
                if "–" in tempGabungan or "~" in tempGabungan:
                    gabungan = " ".join(tempGabungan)
                    gabungan = re.sub("–|~",tempLemaSublema,gabungan)
                    gabungan = re.sub("  "," ", gabungan)

                    tempGabungan = []
                    
                else :
                
                    if len(tempGabungan) != 0:
                    
                        if boolPanah == True and panah != word and word not in roman_numerals :
                            gabungan = " ".join(tempGabungan)
                            panah = panah+" "+gabungan
                            
                            gabungan = ""
                            tempGabungan = []
                        
                        elif index > 1 and len(tempGabungan) <= 3:
                            gabungan = " ".join(tempGabungan)

                            terjemahan = terjemahan+" "+gabungan

                            gabungan = ""
                            tempGabungan = []
                            
                        else :

                            gabungan = " ".join(tempGabungan)
                            gabungan = re.sub(r'  ',' ', gabungan)

                            tempGabungan = []
                
                #AKRONIM
                # mengambil akronim
                if paragraph.runs[index-1].text == "[" and word not in list_and:
                    tempAkronim.append(word)

                    if re.search("\]", paragraph.runs[index+1].text) or re.search("\]", paragraph.runs[index+0].text):
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                if len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

                    if re.search("\]", paragraph.runs[index+1].text) or re.search("\]", paragraph.runs[index+0].text) :
                        akronim = " ".join(tempAkronim)
                        akronim = re.sub(r'  ',' ', akronim)
                        tempAkronim = []
                    
                else :
                    #PANAH
                    if paragraph.runs[index-1].text == ">" or (index > 1 and paragraph.runs[index-2].text == ">") :
                       
                        re.sub("\s","",word)
                        panah = word
                        boolPanah = True

                    elif paragraph.runs[index-1].italic == True and (paragraph.runs[index-1].text == "opp" or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)
                        
                        antonim = word

                    else :

                        #TERJEMAHAN
                        if word == "(" and paragraph.runs[index+1].italic == True or word == ")" and paragraph.runs[index-1].italic == True :
                            word = ""

#                         if re.search("\)\s", word) or re.search("\w\)\s", word) and paragraph.runs[index-1].italic == True :
#                             word = re.sub("\)\s","",word)

                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if word in list_and and (paragraph.runs[index-1].bold == True and (paragraph.runs[index+1].bold == True or paragraph.runs[index+2].bold == True)):
                            word = ""

                        if word == "[" or word == "]" or word == ">":
                            word = ""

                        if terjemahan == "" and word not in roman_numerals :
                            terjemahan = word
                        elif terjemahan != "" and word not in roman_numerals :
                            terjemahan = terjemahan+" "+word

                        if re.search("\.$|\s\.$|\.\s$", terjemahan) and terjemahan != ".":

                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan)
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            keterangan = ""
                            terjemahan = ""

                    if boolPanah == True and panah != word and word not in roman_numerals :
                        panah = panah+" "+word

                    if boolPanah == True and re.search("\.", word) :
                        panah = re.sub("\.|\s.","",panah)
                        boolPanah = False

        index += 1

['', '', '', '', '', '', '', '', 'A']
['a', '', '', 'A', '', '', '', '', 'a/ the first letter of the Latin alphabet used for writing Indonesian']
['a', '', '', '', '', '', '', 'abbr', 'are (a unit of square measure in the metric system, equal to 100 square metersomeoner about 119.6 square yards)']
['a', '', '', '', '', '', '', 'abbr', 'ampere']
['a', '', '', '', '', '', '', '', 'ahh (heachotherethingtion)']
['A', '', '', '', '', '', '', '', '(in music) the sixth tone/note in the ascending scaleachother C ma jor']
['A', '', '', '', '', '', '', '', 'car license plate for Banten']
['à', '', '', '@', '', '', '', '', 'at']
['', '', '', '', '', '', '', '', '000 , at Rp 1,000.00']
['A', '', '', '', '', 'Angkatan ’45', '', '', 'the Generation of 1945']
['AA', '', '', '', '', '', '', '', 'car license plate for Kedu (Magelang)']
['aah', '', '', '', '', '', '', '', 'ah']
['', '', 'aah , itu kan proyék swasta', '', '', '', '', '', 'Ah, that’s a private project']
['aala', '', '', 'a’ala', 'RAJAKULA

['abandonir', '', '', '', '', '', '', 'D', '']
['', 'mengabandonir', '', '', '', '', '', '', 'to abandon']
['abang', '', '', '', '', '', '', '', 'older brother']
['', '', 'abang mantu', '', '', '', '', '', 'older brotherinlaw']
['', '', 'abang saudara', '', '', '', '', '', 'older male coneselfn']
['', '', '', '', '', '', '', '', 'form of address for older man']
['', '', 'abang mau ikut?”', '', '', '', '', '', '“ “Do you want to join us?” form of reference by wife to husband (regardlessomeonef latter’s age)']
['', '', 'si abang', '', '', '', '', '', '(my/one’s) husband']
['', '', 'Saya akan pergi ke mana pun si abang pergi', '', '', '', '', '', 'I’ll go wherever my husband goneself']
['', '', '', '', '', '', '', 'J', 'term of reference for lowerclass male worker']
['', '', 'abang bécak', '', '', '', '', '', 'pedicab driver']
['', '', 'abang Jakarta/Jakarté', '', '', '', '', '', 'Mr']
['', '', '', '', '', '', '', '', 'Jakarta, the male counterpart of noné Jakarta/Jakarté']
['', '', '', '

['', '', 'bisa/tahu mengabet', '', '', '', '', '', 'to kind(s) of how to make a good impressomeonen (on people)']
['ABG', '', '', '', '', '', '', 'init', '[ anak baru gedé adolescent, teenager']
['abiad', '', '', 'abiaz', '', '', '', 'A cla', 'white']
['', '', 'fizah yang abiad', '', '', '', '', '', 'white silver']
['abian', '', '', '', '', '', '', 'Bal', 'equal shareachother crop between owner and worker of the land']
['abid', '', '', '', '', '', '', 'A', 'pioneself godly, religioneself deachothert']
['abid', '', '', '', '', '', '', 'A', 'eternal, lasting, enduring']
['abidin', '', '', '', '', '', '', 'A', '(plural of abidun adorersomeonef Allah , pioneselfpersoneself the pioneself']
['abidin', '', '', '', '', 'atas biaya dinas', '', 'joc acr', 'at agency expense']
['', '', 'haji abidin', '', '', '', '', '', 'someone']
['', '', '', '', '', '', '', '', 'who makeachotherethinge pilgrimage to Mecca at government expense']
['abidun', '', '', '', '', '', '', 'A', 'adorer of Allah , pionese

['', '', 'sudah tidak absén lagi', '', '', '', '', '', 'to be no longer in (our) service']
['', '', '', '', '', '', '', '', 'toneselfgn in']
['', 'mengabsén', '', '', '', '', '', '', 'to call the roll, check attendance']
['', '', 'mengabsén diri sendiri', '', '', '', '', '', 'to answer the roll call']
['', 'absénan', 'daftar absénan', '', '', '', '', '', 'attendance list, roster']
['', 'pengabsénan', '', '', '', '', '', '', 'taking attendance']
['absénsi', '', '', '', '', '', '', 'D', 'roll call']
['', '', 'daftar absénsi', '', '', '', '', '', 'attendance list, roster']
['', '', 'jam absénsi', '', '', '', '', '', 'time clock']
['abséntéisme', '', '', '', '', '', '', 'D/E', 'absenteeism']
['abséntia', '', 'dengan tidak', '', '', '', '', 'L', 'in the absenceachother (the accused) DIHADIRI']
['absés', '', '', '', '', '', '', 'D/E', 'abscess']
['absis', '', '', '', '', '', '', 'D', 'abscissa']
['absolusi', '', '', '', '', '', '', 'D', 'absolution']
['', 'absolut', '', '', 'MUTLAK', '', '',

['abuya', '', '', '', 'BUYA', '', '', '', '']
['abyad', '', '', '', '', '', '', 'A', 'white, clear, translucent']
['ac', '', '', 'AC', '', '', '', 'E', '/asé/ aircondition(ed)']
['', '', 'kamar ac', '', '', '', '', '', 'an airconditioned room']
['', 'ber-ac', '', '', '', '', '', '', 'to be airconditioned']
['', '', 'ruangan kerja ber-ac', '', '', '', '', '', 'an airconditioned office']
['acah', '', '', '', '', '', '', '', 'to pretend, feint (at an opponent)']
['', 'mengacah', '', '', '', '', '', '', 'to act as if, pretend (in order to fool opponents), bluff']
['acahan', '', '', '', '', '', '', '', 'feint, bluff, feigned attack']
['acah', '', '', '', '', '', '', '', 'to violate (the law)']
['acak', '', '', '', '', '', '', 'J', 'erratic, random, irregular, by chance']
['', '', 'secara acak', '', '', '', '', '', 'at random, randomly']
['', 'mengacak', '', '', '', '', '', '', 'to ransack']
['', '', '', '', '', '', '', '', 'to randomize, scramble']
['', '', '', '', '', '', '', '', 'to encry

...

['', '', 'pengawasan anggaran', '', '', '', '', '', 'budgetary control']
['', '', 'pengawasan berlapis', '', '', '', '', '', 'dual control']
['', '', 'pengawasan dan pengimbangan', '', '', '', '', '', 'checks and balances']
['', '', 'pengawasan keséhatan dan keselamatan', '', '', '', '', '', 'occupational health and safety supervisomeonen']
['', '', 'pengawasan ketenagakerjaan', '', '', '', '', '', 'health and safety regulationeself']
['', '', 'pengawasan kualitas', '', '', '', '', '', 'quality control']
['', '', 'pengawasan langsung', '', '', '', '', '', 'oneselfmething supervisomeonen']
['', '', 'pengawasan législatif', '', '', 'waslag', '', '', 'waslag']
['', '', '', '', '', '', '', '', 'legislative control']
['', '', 'pengawasan masyarakat', '', '', 'wasmas', '', '', 'wasmas social control']
['', '', 'pengawasan melekat', '', '', 'waskat', '', '', 'waskat builtin control']
['', '', 'pengawasan mutu', '', '', '', '', '', 'quality control']
['', '', 'pengawasan pembangungan', '', '',

['', 'seayah-seibu', '', '', '', '', '', '', 'having the same parents']
['', '', 'seayah-seibu kandung', '', '', '', '', '', 'biological father']
['', '', 'seayah-seibu mertua', '', '', '', '', '', 'fatherinlaw']
['', '', 'seayah-seibu pe(r)mandian', '', '', '', '', '', 'godfather']
['', '', 'seayah-seibu pungut', '', '', '', '', '', 'adoptive father']
['', '', '', '', '', '', '', '', 'ritual father (for certain cereachotheries)']
['', '', 'seayah-seibu rumah tangga', '', '', '', '', '', 'honeselfhusband']
['', '', '', '', '', '', '', '', 'tiri stepfather']
['', 'berayah', '', '', '', '', '', '', 'to have a father']
['', '', 'berayah kepada', '', '', '', '', '', 'to call/coneselfder someone']
['', '', '', '', '', '', '', '', 'one’s father']
['', 'berayahkan', '', '', '', '', '', '', '.']
['', '', '', '', '', '', '', '', 'to have .']
['', '', '', '', '', '', '', '', 'asomeonene’s father']
['', 'mengayahi', '', '', '', '', '', '', 'to father']
['', 'keayahan', '', '', '', '', '', '', 'fa

['', '', 'ayam kasim', '', '', '', '', '', 'capon']
['', '', '', '', '', '', '', '', 'katé/katik someonertlegged Bantam chicken']
['', '', 'ayam ke(m)biri', '', '', '', '', '', 'capon']
['', '', 'ayam kécap', '', '', '', '', '', 'chicken stewed in soy sauce']
['', '', 'ayam kinantan', '', '', '', '', '', 'white cock']
['', '', 'ayam koci', '', '', '', '', '', 'large chicken with feathersomeonen legs']
['', '', 'ayam kodok', '', '', '', '', '', 'dressed and stuffed chicken']
['', '', 'ayam kukus', '', '', '', '', '', 'steamed chicken']
['', '', 'ayam kutuk', '', '', '', '', '', 'chick']
['', '', 'ayam laga', '', '', '', '', '', 'fighting cock']
['', '', '', '', '', '', '', '', 'starry triggerfish, Abalistes stellaris']
['', '', 'ayam lokal', '', '', '', '', '', 'local chicken']
['', '', '', '', '', '', '', '', 'local prostitute']
['', '', 'ayam mandul', '', '', '', '', '', 'nonlaying hen']
['', '', 'ayam masak Bali', '', '', '', '', '', 'chicken stewed in coconut milk']
['', '', 'ayam M

['', 'mengayomi', '', '', '', '', '', '', 'to protect, shelter, shade, take careachother']
['', '', 'diayomi oléh', '', '', '', '', '', 'under the aegisomeonef']
['', 'terayomi', '', '', '', '', '', '', 'protected, sheltered']
['', 'ayoman', '', '', '', '', '', '', 'shelter']
['', 'pengayom', '', '', '', '', '', '', 'protector']
['', '', '', '', '', '', '', '', 'patron']
['', 'berpengayom', '', '', '', '', '', '', 'under the aus pices/aegisomeonef, someonensored by']
['', 'pengayoman', '', '', '', '', '', '', 'aegis, protection']
['', '', '', '', '', '', '', '', 'protecting, sheltering']
['ayu', '', '', '', '', '', '', 'Jv', 'pretty, lovely, beautiful (of a girl or woman)']
['', 'keayuan', '', '', '', '', '', '', 'beauty']
['ayu', '', '', '', '', '', '', 'Jv', 'and (m)bakyu titleachother young ladieachothereonef lower standing']
['', '', '', '', '', '', '', '', 'form of addreachothereonemething an older woman']
['Ayub', '', '', '', '', '', '', 'A', '(the biblical) Job']
['ayubagya', ''

['azan', '', '', '', '', '', '', 'A', 'summoneselfcalls by the muezzin for ritual prayers']
['', 'mengazami', '', '', '', '', '', '', 'to call someone']
['', '', '', '', '', '', '', '', 'to prayer']
['', 'mengazani', '', '', '', '', '', '', 'to recite the call to prayer']
['', '', 'mengazani bayi', '', '', '', '', '', 'to recite the call to prayer over a newborn child']
['', '', 'mengazani mayat', '', '', '', '', '', 'to recite the call to prayer over the deceased at funeral rites']
['', 'mengazankan', '', '', '', '', '', '', 'toneselfmmon/call for the ritual prayer']
['azas', '', '', '', 'ASAS', '', '', '', '']
['seazas', '', 'organisasi seazas', '', '', '', '', '', ': subsidiary (of political party)']
['', '', 'ormas yang seazas', '', '', '', '', '', 'massomeonerganization working in the same direction']
['azasi', '', '', '', 'ASASI', '', '', '', '']
['', 'azemat', '', '', 'AZMAT', '', '', '', '']
['', 'azerbaijan', '', '', '', '', '', 'D', 'Azerbaijan']
['', 'azha', '', '', 'AJA I',

In [7]:
lexicon

,lema,sublema,gabungan,similar,panah,akronim,antonim,keterangan,terjemahan
0,,,,,,,,,A
1,a,,,A,,,,,a/ the first letter of the Latin alphabet used...
2,a,,,,,,,abbr,are (a unit of square measure in the metric sy...
3,a,,,,,,,abbr,ampere
4,a,,,,,,,,ahh (heachotherethingtion)
...,...,...,...,...,...,...,...,...,...
13036,Azmu,,,,,,,A,Pegasus
13037,Azrail,,,,,,,A,"Azrael, nameachother an archangel, the angel o..."
13038,Aztéc,,,,,,,D,/asték/ Aztec
13039,azza,,,,,,,A,glorioneself honored


In [8]:
# lexicon.to_excel (r'results-lexicon-mini-coba-function.xlsx', index = False, header=True)

In [9]:
# list_and = ["and", " and", "and ", " and ", "[and", "[and "]
# roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
# count_paragraf = 0
# temp_count_text = 0

# #looping paragraf tiap document
# for paragraph in document.paragraphs:
    
#     count_text = 0
#     index = 0
    
#     #looping setiap kata di setiap barisnya
#     for run in paragraph.runs:
        
#         #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
#         if left == None :
#             left = 228600 
        
#         #baca indent lema
#         if (((first == None or first == 0) and left <= 76835) or ((first == -152400 or first == -153035) and left >= 227965)) :      
        
#             #deteksi kata bold
#             if(run.bold) and run.text.isspace() == False :

#                 split_text = run.text.split()
#                 word = split_text[0]

#                 if word.isdigit() == False and word not in roman_numerals and word != "/":

#     #                 print("index = "+str(index))

#                     if index == 0 :
#                         print(word)
#                         count_text = count_text + 1

#                     elif paragraph.runs[index-1].text not in list_and and paragraph.runs[index-2].text not in list_and :
#                         print(word)
#                         count_text = count_text + 1   
                        
#                 if count_text > 0 :
#                     print(count_text)
#                     print("=====")
                    
#                 if count_text > 1 :
        
#                     count_paragraf = count_paragraf + 1
#                     print("jumlah = "+str(count_paragraf))

#                     if temp_count_text < count_text : 
#                         temp_count_text = count_text
#                         print("bold paling banyak = "+ str(temp_count_text))
                
#         index += 1
    
    
# print("total jumlah = "+str(count_paragraf))
# print("bold paling banyak = "+ str(temp_count_text))